In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Get the data path
data_path = Path.cwd().parent.resolve()/"data"/"GFSMAB"/"GFSMAB2015"

# Define the file name
file_name = "GFSMAB2015_01-16-2025.csv"

# Combine the base path and file name
file_path = data_path / file_name

if file_path.exists():  # Check if the file exists
    data = pd.read_csv(file_path)
    print("File loaded successfully.")
else:
    print(f"File not found at {file_path}")


File loaded successfully.


In [36]:
def filter_and_combine(data, column_name, keywords, output_filename):
    """
    Filters rows based on multiple keywords in a specified column and combines them into one CSV.
    
    Parameters:
        data (pd.DataFrame): The DataFrame to search within.
        column_name (str): The column to search for the keywords.
        keywords (list): A list of keywords to search for.
        output_filename (str): The filename to save the combined filtered rows.

    Returns:
        pd.DataFrame: Combined DataFrame of filtered rows.
    """
    combined_rows = pd.DataFrame()  # Initialize an empty DataFrame

    for keyword in keywords:
        # Filter rows where the keyword appears in the specified column (case-insensitive)
        filtered_rows = data[data[column_name].str.contains(keyword, case=False, na=False)]
        
        # Add a column to indicate the keyword used for filtering
        filtered_rows['Keyword'] = keyword
        
        # Append to the combined DataFrame
        combined_rows = pd.concat([combined_rows, filtered_rows], ignore_index=True)

    # Save the combined rows to a single CSV file
    combined_rows.to_csv(Path.cwd() / output_filename, index=False)
    
    # Return the combined DataFrame
    return combined_rows

# Define the DataFrame (assuming `data` is already loaded)
column_name = "Classification Name"

# List of keywords to search for
keywords = ["debt", "liabilities", "borrowing"]

# Call the function to filter and combine rows
output_filename = "filtered_rows_combined.csv"
combined_data = filter_and_combine(data, column_name, keywords, output_filename)

# Display the unique values in the "Classification Name" column
unique_classification_names = combined_data[column_name].unique()
print("Unique classification names:", unique_classification_names)

In [45]:
def analyze_data_format(data):
    """
    Analyzes the dataset to calculate the sum of legitimate entries and covered countries
    for each combination of Classification, Sector, and Unit.
    
    Parameters:
        data (pd.DataFrame): The input dataset.

    Returns:
        pd.DataFrame: Summary DataFrame with analysis results.
    """
    # Filter rows where Attribute is "Value"
    filtered_data = data[data['Attribute'] == 'Value']

    # Select year columns (assume all columns except metadata and Attribute are years)
    year_columns = [col for col in data.columns if col.startswith('20') or col.startswith('19')]

    # Group by Classification, Sector, and Unit
    grouped = filtered_data.groupby(
        ['Classification Name', 'Sector Name', 'Unit Name']
    )

    # Calculate the required summaries
    summary = grouped.apply(
        lambda group: pd.Series({
            'Sum of Legitimate Entries': group[year_columns].apply(
                pd.to_numeric, errors='coerce'
            ).count().sum(),
            'Number of Covered Countries': group['Country Code'].nunique()
        })
    ).reset_index()

    return summary

# Assuming your dataset is loaded in `data`
summary = analyze_data_format(combined_data)

# Save the summary to a CSV file if needed
summary.to_csv(Path.cwd().parent.resolve()/"data"/"GFSYMAB"/"Summaries"/"summary_analysis_2015.csv", index=False)


/tmp/ipykernel_66411/4276670487.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = grouped.apply(


In [2]:
import pandas as pd
from pathlib import Path

# Define the data directory
data_dir = Path.cwd().parent.resolve() / "data" / "GFSMAB"
output_dir = data_dir / "Summaries"

# Ensure the output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# List of years to process
years = [2014, 2015, 2016, 2017, 2019, 2020]

# List of keywords for filtering
keywords = ["debt", "liabilities", "borrowing"]

def load_data_for_year(data_dir, year):
    """Load the CSV file for a specific year."""
    file_dir = f"GFSMAB{year}"
    file_name = f"GFSMAB{year}_01-16-2025.csv"
    file_path = data_dir / file_dir / file_name
    if file_path.exists():
        return pd.read_csv(file_path)
    else:
        print(f"File not found for year {year}: {file_path}")
        return None

def filter_and_combine(data, column_name, keywords):
    """Filter rows based on multiple keywords and return combined results."""
    combined_rows = pd.DataFrame()
    for keyword in keywords:
        filtered_rows = data[data[column_name].str.contains(keyword, case=False, na=False)]
        filtered_rows['Keyword'] = keyword
        combined_rows = pd.concat([combined_rows, filtered_rows], ignore_index=True)
    return combined_rows

def analyze_data_format(data):
    """Analyze data to calculate sums of legitimate entries and covered countries."""
    filtered_data = data[data['Attribute'] == 'Value']
    year_columns = [col for col in data.columns if col.startswith(('20', '19'))]
    grouped = filtered_data.groupby(['Classification Name', 'Sector Name', 'Unit Name'])
    summary = grouped.apply(
        lambda group: pd.Series({
            'Sum of Legitimate Entries': group[year_columns].apply(pd.to_numeric, errors='coerce').count().sum(),
            'Number of Covered Countries': group['Country Code'].nunique()
        })
    ).reset_index()
    return summary

def process_all_years(data_dir, years, keywords, output_dir):
    """Process all years, filter and analyze data, and save summaries."""
    for year in years:
        # Load data
        data = load_data_for_year(data_dir, year)
        if data is None:
            continue  # Skip if the file for the year is not found

        # Filter and combine data
        combined_data = filter_and_combine(data, "Classification Name", keywords)

        # Analyze data
        summary = analyze_data_format(combined_data)

        # Save the summary
        summary_file = output_dir / f"summary_analysis_{year}.csv"
        summary.to_csv(summary_file, index=False)
        print(f"Summary for {year} saved to {summary_file}")

# Run the processing
process_all_years(data_dir, years, keywords, output_dir)


/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Summary for 2014 saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/Summaries/summary_analysis_2014.csv


/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Summary for 2015 saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/Summaries/summary_analysis_2015.csv


/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

...

/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Summary for 2019 saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/Summaries/summary_analysis_2019.csv


/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Keyword'] = keyword
/tmp/ipykernel_97024/3508515484.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Summary for 2020 saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/Summaries/summary_analysis_2020.csv


/tmp/ipykernel_97024/3508515484.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = grouped.apply(


In [3]:
import pandas as pd
from pathlib import Path

# Define the directory containing summary files
summary_dir = output_dir

# Load all summary files into a list of DataFrames
summary_dfs = []
for year in years:
    file_path = summary_dir / f"summary_analysis_{year}.csv"
    if file_path.exists():
        summary_dfs.append(pd.read_csv(file_path))
    else:
        print(f"Summary file for {year} not found at {file_path}")

# Combine all summaries into a single DataFrame
combined_summary = pd.concat(summary_dfs, ignore_index=True)

# Group by relevant columns and calculate aggregated values
aggregated_summary = combined_summary.groupby(
    ['Classification Name', 'Sector Name', 'Unit Name'],
    as_index=False
).agg({
    'Sum of Legitimate Entries': 'sum',
    'Number of Covered Countries': 'max'
})

# Sort by the most legitimate entries
aggregated_summary = aggregated_summary.sort_values(by='Sum of Legitimate Entries', ascending=False)

# Save the aggregated summary to a CSV file
output_path = summary_dir / "aggregated_summary_2014_2020.csv"
aggregated_summary.to_csv(output_path, index=False)

print(f"Aggregated summary saved to {output_path}")


Aggregated summary saved to /home/torbenhaferkamp/Desktop/IfW_Kiel/GSF/hidden_debt_gsf/src/hidden_debt_gsf/data/GFSMAB/Summaries/aggregated_summary_2014_2020.csv
